In [22]:
import sqlite3
import pandas as pd

conn=sqlite3.connect(r'C:\Users\munnaboy\Desktop\chinook.db')

tables = pd.read_sql("""
    SELECT name 
    FROM sqlite_master 
    WHERE type='table';
""", conn)

print(tables)



cs=pd.read_sql("SELECT * FROM Invoice",conn)
print(cs.head())


query ="""
Select c.FirstName,c.LastName,sum(i.Total) as TotalSpent
From Invoice i
Join Customer c on i.CustomerId=c.CustomerId
Group by c.FirstName,c.LastName
order by TotalSpent desc
"""

df=pd.read_sql(query,conn)
print(df.head())



             name
0           Album
1          Artist
2        Customer
3        Employee
4           Genre
5         Invoice
6     InvoiceLine
7       MediaType
8        Playlist
9   PlaylistTrack
10          Track
   InvoiceId  CustomerId          InvoiceDate           BillingAddress  \
0          1           2  2009-01-01 00:00:00  Theodor-Heuss-Straße 34   
1          2           4  2009-01-02 00:00:00         Ullevålsveien 14   
2          3           8  2009-01-03 00:00:00          Grétrystraat 63   
3          4          14  2009-01-06 00:00:00           8210 111 ST NW   
4          5          23  2009-01-11 00:00:00          69 Salem Street   

  BillingCity BillingState BillingCountry BillingPostalCode  Total  
0   Stuttgart         None        Germany             70174   1.98  
1        Oslo         None         Norway              0171   3.96  
2    Brussels         None        Belgium              1000   5.94  
3    Edmonton           AB         Canada           T6G 2C7   8

In [31]:
query_2 = """
SELECT c.CustomerId, SUM(i.Total) AS TotalPurchase
FROM Customer c 
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalPurchase DESC
LIMIT 5;
"""

df = pd.read_sql(query_2, conn)
print(df.head())



   CustomerId  TotalPurchase
0           6          49.62
1          26          47.62
2          57          46.62
3          45          45.62
4          46          45.62


In [33]:
query_3 = """
SELECT c.CustomerId, c.FirstName, c.LastName, SUM(i.Total) AS TotalPurchase
FROM Customer c 
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId, c.FirstName, c.LastName
ORDER BY TotalPurchase DESC
LIMIT 5;
"""

df = pd.read_sql(query_3, conn)
print(df)


   CustomerId FirstName    LastName  TotalPurchase
0           6    Helena        Holý          49.62
1          26   Richard  Cunningham          47.62
2          57      Luis       Rojas          46.62
3          45  Ladislav      Kovács          45.62
4          46      Hugh    O'Reilly          45.62


In [51]:
tr =pd.read_sql("Select * From Track",conn)
print(tr.head())


artist = """
WITH FullAlbumPurchase AS (
    SELECT t.AlbumId, il.InvoiceId, COUNT(DISTINCT il.TrackId) AS TracksBought
    FROM InvoiceLine il
    JOIN Track t ON il.TrackId = t.TrackId
    GROUP BY t.AlbumId, il.InvoiceId
    HAVING TracksBought = (
        SELECT COUNT(*) FROM Track t2 WHERE t2.AlbumId = t.AlbumId
    )
)

SELECT 
    ROUND(
        100.0 * COUNT(DISTINCT ar.ArtistId) /
        (SELECT COUNT(*) FROM Artist),
    2
    ) AS PercentArtistsWithFullAlbumSales
FROM Album al
JOIN Artist ar ON al.ArtistId = ar.ArtistId
JOIN FullAlbumPurchase f ON f.AlbumId = al.AlbumId;


"""

df=pd.read_sql(artist,conn)
print(df.head())


   TrackId                                     Name  AlbumId  MediaTypeId  \
0        1  For Those About To Rock (We Salute You)        1            1   
1        2                        Balls to the Wall        2            2   
2        3                          Fast As a Shark        3            2   
3        4                        Restless and Wild        3            2   
4        5                     Princess of the Dawn        3            2   

   GenreId                                           Composer  Milliseconds  \
0        1          Angus Young, Malcolm Young, Brian Johnson        343719   
1        1                                               None        342562   
2        1  F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...        230619   
3        1  F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...        252051   
4        1                         Deaffy & R.A. Smith-Diesel        375418   

      Bytes  UnitPrice  
0  11170334       0.99  
1   5510424 

In [55]:
tracks = """
WITH AlbumTrackCounts AS (
    SELECT AlbumId, COUNT(*) AS TotalTracks
    FROM Track
    GROUP BY AlbumId
),

AlbumPurchases AS (
    SELECT il.InvoiceId, t.AlbumId, i.CustomerId, COUNT(DISTINCT il.TrackId) AS TracksBought
    FROM InvoiceLine il
    JOIN Track t ON il.TrackId = t.TrackId
    JOIN Invoice i ON il.InvoiceId = i.InvoiceId  
    GROUP BY il.InvoiceId, t.AlbumId
),

FullAlbumBuyers AS (
    SELECT ap.CustomerId
    FROM AlbumPurchases ap
    JOIN AlbumTrackCounts atc ON ap.AlbumId = atc.AlbumId
    WHERE ap.TracksBought = atc.TotalTracks
    GROUP BY ap.CustomerId
),

AllBuyers AS (
    SELECT DISTINCT CustomerId FROM Invoice
)

SELECT ab.CustomerId
FROM AllBuyers ab
LEFT JOIN FullAlbumBuyers fab ON ab.CustomerId = fab.CustomerId
WHERE fab.CustomerId IS NULL

"""

td = pd.read_sql(tracks,conn)
print(td.head())



   CustomerId
0           5
1           6
2           8
3           9
4          10


In [57]:


final = """
WITH FullAlbumBuyers AS (
    SELECT DISTINCT i.CustomerId
    FROM InvoiceLine il
    JOIN Track t ON il.TrackId = t.TrackId
    JOIN Invoice i ON il.InvoiceId = i.InvoiceId
    JOIN (
        SELECT AlbumId, COUNT(*) AS TotalTracks
        FROM Track
        GROUP BY AlbumId
    ) atc ON t.AlbumId = atc.AlbumId
    GROUP BY i.InvoiceId, t.AlbumId
    HAVING COUNT(DISTINCT il.TrackId) = atc.TotalTracks
),

AllBuyers AS (
    SELECT DISTINCT CustomerId FROM Invoice
)

SELECT
    (SELECT COUNT(*) FROM FullAlbumBuyers) AS FullAlbumBuyers,
    (SELECT COUNT(*) FROM AllBuyers) - (SELECT COUNT(*) FROM FullAlbumBuyers) AS IndividualTrackBuyers,
    (SELECT COUNT(*) FROM AllBuyers) AS TotalCustomers,
    ROUND(100.0 * (SELECT COUNT(*) FROM FullAlbumBuyers) / (SELECT COUNT(*) FROM AllBuyers), 2) AS FullAlbumPercent,
    ROUND(100.0 * ((SELECT COUNT(*) FROM AllBuyers) - (SELECT COUNT(*) FROM FullAlbumBuyers)) / (SELECT COUNT(*) FROM AllBuyers), 2) AS IndividualTrackPercent;
"""

fq =pd.read_sql(final,conn)
print(fq.head())

   FullAlbumBuyers  IndividualTrackBuyers  TotalCustomers  FullAlbumPercent  \
0               21                     38              59             35.59   

   IndividualTrackPercent  
0                   64.41  
